In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('Childcare_Data_CA_2023.csv')
df

In [ ]:
df.drop(columns=['01/16/2024 Download Date\nChild Care Center Lead\nSampling Results','LabNumber','XMOD','RegOffice'])

In [ ]:
id = pd.read_csv('dss_cleaned.csv')
id.drop(columns=['Column'], inplace=True)
id.rename(columns={'Facility Number':'Id'}, inplace=True)


In [ ]:
merged_childcare = df.merge(id, on="Id", how="outer")
merged_childcare.drop(columns=['County', 'City', 'Name'], inplace=True)
merged_childcare

In [ ]:
# Number of tests in each city.

group1 = merged_childcare.groupby(['Facility City'])['Result'].count().reset_index()
total_tests = group1['Result']

#  Median test result

group2 = merged_childcare.groupby(['Facility City'])['Result'].median().round(5).reset_index()
median_result = group2['Result']

# Total number of facilities per city.

total_facilities = merged_childcare.groupby(['Facility City'])['Id'].nunique().reset_index(name='total_facilities')

# Total number of facilities with a test.

tested_facilities = merged_childcare[merged_childcare.Result.notnull()].groupby('Facility City')['Id'].nunique().reset_index(name='tested_facilities')

# Number of tests in each city over 5ppb and 15ppb

df_over5 = merged_childcare.groupby('Facility City')['Result'].apply(lambda x: (x>=5).sum()).reset_index(name='count')
df_over15 = merged_childcare.groupby('Facility City')['Result'].apply(lambda x: (x>=15).sum()).reset_index(name='count')

tests_5 = df_over5['count']
tests_15 = df_over15['count']

In [ ]:
final = total_facilities.merge(tested_facilities, on='Facility City', how='outer', )
final['tested_facilities'].fillna(0, inplace=True)
final['tested_facilities'] = final['tested_facilities'].astype(int)
final['total_tests'] = total_tests
final['median_result'] = median_result
final['tests_5'] = tests_5
final['tests_15'] = tests_15
final